## String testing
* Need to install nltk, textblob, and autocorrect

In [1]:
import re
from collections import Counter


# import nltk
# import numpy as np
import pandas as pd
# from babel.numbers import format_currency
from calitp import *
from nltk import ngrams
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
# from sklearn.feature_extraction.text import TfidfVectorizer
# import ast
from autocorrect import Speller
from textblob import TextBlob

In [2]:
import A1_data_prep
import A2_tableau
import A8_strings

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.


In [3]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [4]:
df_tircp = to_snakecase(A2_tableau.tableau_dashboard())

/opt/conda/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed


In [5]:
df_tircp = A8_strings.simplify_descriptions(
    df_tircp, "description", "clean_description", A8_strings.description_words_to_delete
)

### Extract Numbers
* Goal: get number of "whatever" purchased.
* Remove adjectives from description 

#### Try to extract all the numbers before a certain keyword
* https://stackoverflow.com/questions/69741399/regex-extract-numbers-preceded-by-defined-strings

In [6]:
my_keywords_list = [A8_strings.bus_list, A8_strings.other_vehicles_list, A8_strings.lrv_list, A8_strings.train_list, A8_strings.other_list]
my_new_column_names_list = ["bus", "other vehicles", "lrv", "train", "other"]

In [7]:
df_test = A8_strings.total_procurement_estimates(df_tircp,
    "clean_description",
    my_keywords_list,
    my_new_column_names_list,)

In [8]:
df_test = df_test[['award_year','title','description', 'clean_description','total_bus','total_other vehicles', 'total_lrv', 'total_train','total_other']]

In [23]:
# df_test[df_test.total_train > 0]

In [9]:
df_test[['total_bus','total_other vehicles', 'total_lrv', 'total_train',]].sum()

total_bus               737
total_other vehicles     94
total_lrv                31
total_train             355
dtype: int64

### Categorizing Projects
* Categorize projects in general
* Find most common phrases out of descriptions_list created above in Natalie's function
* https://stackoverflow.com/questions/60037924/how-count-the-most-frequently-repeated-phrases-in-pandas

In [10]:
# Place common keywords into a dataframe
# https://stackoverflow.com/questions/64593557/how-to-find-most-common-word-from-the-entire-column-of-string-in-python
# descriptions_list = get_list_of_words(df_test, "description")

In [11]:
# counter = Counter()  # Initializing a counter variable

In [12]:
#for keyword in descriptions_list:
#    split_string = re.findall(r"\w+", keyword)
#    counter.update(split_string)

In [13]:
# Returns the word and how many times it appears
# Ex: service appears 75 times, hydrogen 5, realiability: 4. 
# most_common = counter.most_common

# Turn into a dictionary by copying and pasting everything
# Counter({'blah':3, 'blah',34}) 

In [14]:
# Make a dataframe out of the dictionary
"""
df_common_words = (
    pd.DataFrame.from_dict(most_common_dict, orient="index")
    .reset_index()
    .rename(columns={"index": "word", 0: "total apperance"})
)
"""

'\ndf_common_words = (\n    pd.DataFrame.from_dict(most_common_dict, orient="index")\n    .reset_index()\n    .rename(columns={"index": "word", 0: "total apperance"})\n)\n'

In [15]:
my_additional_keywords = (
    "zero emission",
    "capacity improvements",
    "safety",
    "contactless payment",
    "light rail vehicles",
    "increase ridership",
    "new routes",
    "mobile ticketing",
    "service expansion",
    "network integration",
    "expansion",
    "expanded"
)

In [16]:
def find_common_phrases(df, description_column:str, values_to_add: list):
    
    # Break apart every word in the description column into a list
    descriptions_list = A8_strings.get_list_of_words(df, description_column)
    
    # Get phrases of whatever length you want (2,3,4,etc)
    c = Counter([" ".join(y) for x in [2] for y in ngrams(descriptions_list, x)])
    
    # Make a dataframe out of the counter values
    df_phrases = pd.DataFrame({"phrases": list(c.keys()), "total": list(c.values())})
    
    # Take phrases that are the most repeated and turn it into a list
    df_phrases = ((df_phrases.loc[df_phrases["total"] > 7])).reset_index(drop = True)
    
    common_phrases_list = df_phrases.phrases.tolist()
    
    # Add some other categories to common_phrases_list
    common_phrases_list.extend(values_to_add)
    
    # Remove LA
    common_phrases_list.remove('los angeles')
    
    return common_phrases_list 

In [17]:
def categorize_projects(df, description_column:str, values_to_add: list):
    
    # Find most common 2 word phrases for some automatic project categories
    common_phrases_list = find_common_phrases(df,description_column,values_to_add)
    
    # Place all the words in common_phrases_list into a blob named query
    # Ex: 'lrv|zero emission|electric' 
    # https://stackoverflow.com/questions/64727090/extract-all-matching-keywords-from-a-list-of-words-and-create-a-new-dataframe-pa
    query = "|".join(common_phrases_list)
    
    # Remove punctation and lower strings in original description column befores searching
    df["clean_description"] = (df[description_column]
                      .str.lower()
                      .str.replace("-", " ", regex=True)
                      .str.replace("("," ", regex=True)
                      .str.replace(")"," ", regex=True)
                      .str.replace("."," ", regex=True)
                      .str.strip()
                     )
    
    # Search through description column for the most common phrases 
    df["project_category"] = df["clean_description"].str.findall(
    r"\b({})\b".format(query))
    
    # Explode to take categories out of a list
    # Drop duplicate project keywords by title
    df = (df
          .explode("project_category")
          .sort_values(["award_year", "title"])
          .drop_duplicates(subset=["title", "project_category"]))
    
    # Fill any uncategorized projects as "Other"
    df["project_category"] = (df["project_category"].fillna("Other").str.title())
    
    # Summarize - put all the categories onto one line
    df = (df
          .groupby(["title", "description"])["project_category"]
          .apply(",".join)
          .reset_index())
    
    return common_phrases_list, df

In [18]:
my_list, tircp = categorize_projects(df_test, 'description', my_additional_keywords)

In [24]:
tircp

,title,description,project_category
0,ACE Near-Term Capacity Improvement Program\n,length platforms for 8 car trains and purchase tier in locomotive to allow busiest train to add seating capacity,Other
1,#Electrify Anaheim: Changing the Transit Paradigm in Southern California,"deploys 40 zero emission electric buses to double service levels on up to 8 routes, add 2 new routes; implements a new circulatory/on demand first mile/last mile service; and construction of a new maintenance facility with solar canopy structures","Zero Emission,Electric Buses,New Routes"
2,ACE Wayside Power Project,reduce emissions: eliminated the need for overnight dling of diesel engines through installation of aside power sources at ce’s new regional maintenance facility in stockton,Other
3,ATN FAST (Family of Advanced Solutions for Transit): Revolutionizing Transit for a Global Audience,"creates a zero emission transit ecosystem that offers end to end solutions for residents, employees and the global audience drawn by tourism/convention centers and the lt 2028 summer olympics events project components include 1 purchase of 7 zero emission battery electric vans to implement a new service connecting john wayne airport to anthem, 2 purchase of 10 electric vehicles and associated infrastructure to expand on demand micro transit services into new neighborhoods and service areas, 3 purchase of 15 zeroemission buses to replace existing buses and augment existing routes, including installation of photovoltaic electricity generation at two facilities, and 4 purchase of 10 additional zero emission buses for a new east/west connector service","Zero Emission,Zeroemission Buses"
4,Accelerating Rail Modernization and Expansion in the Capital Region,"expanded service to follow combined with previous irc award to allow for 15 min service during weekdays, plus 3 peak express trains in the peak hour direction begins initial effort to replace the existing fleet with low floor rail vehicles loss includes funding 20 expansion and replacement vehicles and an investment in the highest priority platform conversions to allow efficient and accessible boarding to the new vehicles","Expanded,Expansion"
5,Airport Metro Connector 96th Street Station/Metro Green Line Extension to LAX,"construct a new multi modal transit station along the new crenshaw/ax line cla and integrate the green line service via transfers at the century aviation and mc station this project will connect and improve transit service to los angeles international airport ax by joining metro rail automated people mover pm service, metro bus, municipal bus service, and bicycle facilities at a single multi modal station",Bus Service
6,All Aboard 2018: Transforming SoCal Rail Travel,"improve on time performance and rail corridor capacity for pacific surfliner and coaster trains by investing in signal optimization, a more robust capital maintenance program and new right of way fencing these projects prepare the corridor for higher frequency services on the pacific surfliner and coaster also includes study of san diego maintenance/layer facility relocation on time performance with cd",Other
7,All Aboard: Transforming Southern California Rail Travel,lossan wide network integration & strategic investment planning,Network Integration
8,BART Silicon Valley Phase II Extension,art rail service extension to san jose,Other
9,Blue Line Rail Corridor Transit Enhancements,"increased ridership through investments allowing blue line trolley frequency increases and the addition of a new rapid bus service connecting imperial beach and the stay mesa international border crossing for 15 min frequency to the blue line trolley, also includes supplemental funding to acquire eleven, 60 foot articulated zero emission buses, as well as station improvements","Bus Service,Zero Emission"


In [20]:
# tircp.project_category.value_counts()